In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<h2> Data visualisation and preparation </h2>

On commence par importer les données téléchargées préalablement. On regarde ensuite les informations générales concernant le DataFrame obtenu. 

In [ ]:
# load data
data = pd.read_csv('survey.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.nunique()

Les commentaires et dates/heures de réponse au questionnaire n'interviennent à priori pas dans la nature des réponses données. On peut donc supprimer les colonnes correspondantes. 

In [ ]:
data = data.drop('Timestamp', axis=1)

In [ ]:
data = data.drop('comments', axis=1)

<h5>Age </h5>

On s'intéresse à la variable "Age".

In [ ]:
sns.countplot(y="Age", data=data)

In [ ]:
print(data.Age.value_counts())

In [ ]:
data.Age.describe()

On remarque des réponses absurdes, comme des valeurs négatives d'âge (impossible) ou encore des âges supérieurs à 300 (de même, impossible). On décide donc de supprimer les lignes associées à ces réponses. On enlève également les personnes mineures (5, 8 et 11 ans). La moyenne d'âge est de 32 ans.

In [ ]:
#on enlève les réponses absurdes
data = data.drop(data[data.Age < 18].index)
data = data.drop(data[data.Age > 80].index)

Pour faciliter le modèle, on sépare les âges en plusieurs catégories (tranches d'âge). De cette manière, la réponse n'est plus numérique mais uniquement True/False (1 ou 0) selon l'appartenance ou non à une tranche d'âge. On choisit les tranches 18-25 ans (jeunes travailleurs et diplômés), 26-35 ans (un peu moins jeunes, positions intermédiaires), 36-50 ans, et + 50 ans (seniors). On aura ainsi 4 nouvelles variables contenant uniquement des 0 et des 1.

In [ ]:
#on regroupe les âges par "paliers" : 18-25, 26-35, 35-50, +50
list_age_18_25=[]
list_age_26_35=[]
list_age_36_50=[]
list_age_over50=[]
for part in data['Age']:
    #18-25
    if (int(part)>=18) and (int(part)<=25):
        list_age_18_25.append(1)
        list_age_26_35.append(0)
        list_age_36_50.append(0)
        list_age_over50.append(0) 
    #26-35
    elif (int(part)>=26) and (int(part)<=35):
        list_age_18_25.append(0)
        list_age_26_35.append(1)
        list_age_36_50.append(0)
        list_age_over50.append(0) 
    #36-50
    elif (int(part)>=36) and (int(part)<=50):
        list_age_18_25.append(0)
        list_age_26_35.append(0)
        list_age_36_50.append(1)
        list_age_over50.append(0) 
    #>50
    elif (int(part)>50) and (int(part)<=80):
        list_age_18_25.append(0)
        list_age_26_35.append(0)
        list_age_36_50.append(0)
        list_age_over50.append(1) 

In [ ]:
data['18_25']=list_age_18_25
data['25_35']=list_age_26_35
data['36_50']=list_age_36_50
data['Over50']=list_age_over50

On supprime maintenant la variable Age.

In [ ]:
#on supprime age
#data = data.drop('Age', axis=1)

<h5>Gender </h5>

On s'intéresse à la variable "Gender" représentant le genre de la personne interrogée.

In [ ]:
print(data.Gender.value_counts())

Le questionnaire étant ouvert sur cette question, il n'y a pas de réponse "classique". On décide alors de séparer les genres en 3 catégories : Hommes (M), Femmes (F) et Autres (Other). 

In [ ]:
def correct_gender():
    list_gender=[]
    for part in data['Gender']:
        #'male leaning androgynous',
        if part in ['Male', 'male', 'M', 'm', 'Make', 'Cis Male', 'msle', 'Man', 'cis male', 'Malr', 'Cis Man', 'Mail', 'Guy (-ish) ^_^', 'Male-ish', 'maile', 'something kinda male?', 'Mal', 'Male (CIS)', 'ostensibly male, unsure what that really means']:
            list_gender.append('M')
        #'Female (trans)', 'Trans woman', 'Trans-female', 
        elif part in ['Female', 'female', 'F', 'f', 'Woman', 'Female', 'Female (cis)', 'cis-female/femme', 'femail', 'Cis Female', 'Female', 'woman']:
            list_gender.append('F')
        else:
            list_gender.append('Other')
    return list_gender

In [ ]:
list_genders = correct_gender()
data['Gender']=list_genders

In [ ]:
print(data.Gender.value_counts())
sns.countplot(y="Gender", data=data)

In [ ]:
sns.displot(x=data["Gender"],hue="treatment",data=data,kde=False)
plt.title("Gender Distribution");

Après traitement, nous observons un gros déséquilibre dans la représentation des genres sur l’échantillon : M 986 F 244 et Other 21. On trouve plus de 2 fois plus de femmes traitées pour troubles psychologiques que de femmes non traitées, tandis que chez les hommes, on trouve un peu moins de personnes traitées que de non traitées.

In [ ]:
data.Gender.describe()

<h5>Country </h5>

On s'intéresse à la variable "Country" représentant le pays d'origine des personnes ayant répondu au sondage.

In [ ]:
print(data.Country.value_counts())

In [ ]:
data.Country.describe()

On choisit de simplifier en appliquant le même principe que pour l'âge. Nous choisissons donc de séparer les origines en continents. Nous choisissons Europe, Amérique (Nord et Sud), Afrique, Asie et Océanie.
Nous pouvons également remarquer que plus des 2/3 des participants sont originaires des Etats-Unis. 

In [ ]:
list_europe=[]
list_america=[]
list_africa=[]
list_asia=[]
list_oceania=[]
for part in data['Country']:
    #europe
    if part in ['United Kingdom', 'Germany', 'Ireland', 'Netherlands', 'France', 'Poland', 'Switzerland', 'Sweden', 'Italy', 'Belgium', 'Bulgaria', 'Austria', 'Finland', 'Denmark', 'Greece', 'Croatia', 'Portugal', 'Moldova', 'Georgia', 'Czech Republic', 'Norway', 'Romania', 'Hungary', 'Bosnia and Herzegovina', 'Spain', 'Latvia', 'Slovenia']:
        list_europe.append(1)
        list_america.append(0)
        list_africa.append(0)
        list_asia.append(0)
        list_oceania.append(0)
    #america
    elif part in ['United States', 'Mexico', 'Canada', 'Brazil', 'Colombia', 'Uruguay', 'Bahamas, The', 'Costa Rica']:
        list_europe.append(0)
        list_america.append(1)
        list_africa.append(0)
        list_asia.append(0)
        list_oceania.append(0)
    #africa
    elif part in ['Nigeria', 'Zimbabwe']:
        list_europe.append(0)
        list_america.append(0)
        list_africa.append(1)
        list_asia.append(0)
        list_oceania.append(0)
    #asia
    elif part in ['China', 'Philippines', 'India', 'Israel', 'Singapore', 'Russia', 'Thailand', 'Japan']:
        list_europe.append(0)
        list_america.append(0)
        list_africa.append(0)
        list_asia.append(1)
        list_oceania.append(0)
    #oceania
    else:
        list_europe.append(0)
        list_america.append(0)
        list_africa.append(0)
        list_asia.append(0)
        list_oceania.append(1)

In [ ]:
data['Europe']=list_europe
data['America']=list_america
data['Africa']=list_africa
data['Asia']=list_asia
data['Oceania']=list_oceania
#data = data.drop('Country', axis=1)

Pour les personnes ayant répondu et étant originaires des Etats-Unis, une question supplémentaire était proposée. Ainsi, nous avons les Etats (States) des américains ayant participé à ce sondage. 

In [ ]:
sns.countplot(y="state", data=data)

<h5>Other variables </h5>

In [ ]:
sns.countplot(y="self_employed", data=data)

In [ ]:
data.self_employed.describe()

In [ ]:
sns.displot(x=data["self_employed"],hue="treatment",data=data,kde=False)
plt.title("Self Employed Distribution");

Le fait de travailler pour soi-même, à son propre compte en étant indépendant ne semble pas avoir d'effet spécifique sur la prise ou non de traitement. 

In [ ]:
sns.countplot(y="family_history", data=data)

In [ ]:
data.family_history.describe()

In [ ]:
sns.displot(x=data["family_history"],hue="treatment",data=data,kde=False)
plt.title("Family History Distribution");

Nous pouvons remarquer que l'historique familial joue un rôle important dans la prise de traitement. Ainsi, une personne dont la famille a un historique sera plus suceptible de prendre un traitement, et inversement.

In [ ]:
sns.countplot(y="treatment", data=data)

In [ ]:
data.treatment.describe()

In [ ]:
sns.countplot(y="work_interfere", data=data)

In [ ]:
data.work_interfere.describe()

In [ ]:
sns.displot(x=data["work_interfere"],hue="treatment",data=data,kde=False)
plt.title("Work Interfere Distribution");

De manière logique les personne traitées sont plus à même d'être gênées dans leur travail par des troubles psychologiques.

In [ ]:
sns.countplot(y="no_employees", data=data)

In [ ]:
data.no_employees.describe()

In [ ]:
sns.displot(x=data["no_employees"],hue="treatment",data=data,kde=False)
plt.xticks(rotation=45)
plt.title("Number Employees Distribution");

Il semblerait que le nombre d'employés ait un effet sur la prise ou non de traitement. Cela peut s'expliquer par la sensation d'utilité au sein de l'entreprise, la proximité avec les managers, la pression... 

In [ ]:
sns.countplot(y="remote_work", data=data)

In [ ]:
data.remote_work.describe()

In [ ]:
sns.displot(x=data["remote_work"],hue="treatment",data=data,kde=False)
plt.title("Remote Work Distribution");

Il ne semble pas y avoir de lien très fort entre le fait de travailler en télétravail ou non et la prise de traitement. Les deux semblent à peu près équivalents. 

In [ ]:
sns.countplot(y="tech_company", data=data)

In [ ]:
data.tech_company.describe()

In [ ]:
sns.displot(x=data["tech_company"],hue="treatment",data=data,kde=False)
plt.title("Tech Company Distribution");

De même, la nature de l'entreprise (tech ou non) ne semble pas avoir beaucoup d'effet sur la prise d'un traitement. 

In [ ]:
sns.countplot(y="benefits", data=data)

In [ ]:
data.benefits.describe()

In [ ]:
sns.displot(x=data["benefits"],hue="treatment",data=data,kde=False)
plt.title("Benefits Distribution");

On peut remarquer que lorsque l'entreprise propose des aides au niveau de la santé mentale (variable benefits), les employés sont plus à même de prendre un traitement. On peut l'expliquer par le fait que l'accès à ces soins est facilité et probablement moins cher pour les pays ne disposant pas de sécurité sociale réelle, comme les Etats-Unis (desquels 2/3 des participants sont originaires).

In [ ]:
sns.countplot(y="care_options", data=data)

In [ ]:
data.care_options.describe()

In [ ]:
sns.displot(x=data["care_options"],hue="treatment",data=data,kde=False)
plt.title("Care Options Distribution");

De la même manière, lorsque l'entreprise propose des options d'accompagnement au niveau de la santé mentale (variable care_options), les employés sont plus à même de prendre un traitement. L'explication à ce phénomène est la même que pour la variable benefits. 

In [ ]:
sns.countplot(y="wellness_program", data=data)

In [ ]:
data.wellness_program.describe()

In [ ]:
sns.displot(x=data["wellness_program"],hue="treatment",data=data,kde=False)
plt.title("Wellness Program Distribution");

Pour wellness_program, on retrouve le même schéma dans de plus petites proportions. Cette variable représente la présence d'un point sur la santé mentale dans les "packages" visant au bien-être des employés. 

In [ ]:
sns.countplot(y="seek_help", data=data)

In [ ]:
data.seek_help.describe()

In [ ]:
sns.displot(x=data["seek_help"],hue="treatment",data=data,kde=False)
plt.title("Seek Help Distribution");

Cette variable représente la présence ou non de ressources éduquant les problèmes de santé mentale dans le cadre du travail. On peut l'expliquer de la même manière que précédemment.

In [ ]:
sns.countplot(y="anonymity", data=data)

In [ ]:
data.anonymity.describe()

In [ ]:
sns.displot(x=data["anonymity"],hue="treatment",data=data,kde=False)
plt.title("Anonymity Distribution");

Cette variable représente le fait de pouvoir rester anonyme lorsqu'un employé décide d'utiliser les moyens mis à sa disposition par l'entreprise par rapport à la santé mentale. 

In [ ]:
sns.countplot(y="leave", data=data)

In [ ]:
data.leave.describe()

In [ ]:
sns.displot(x=data["leave"],hue="treatment",data=data,kde=False)
plt.xticks(rotation=45)
plt.title("Leave Distribution");

Cette variable représente la facilité de prise de congé pour des raisons de santé mentale. On remarque que lorsque la prise de ces congés est difficile, la prise de traitement est plus forte. En revanche, elle n'est pas plus faible lorsque la prise de congé est simple. 

In [ ]:
sns.countplot(y="mental_health_consequence", data=data)

In [ ]:
data.mental_health_consequence.describe()

In [ ]:
sns.displot(x=data["mental_health_consequence"],hue="treatment",data=data,kde=False)
plt.title("Mental Health Consequence Distribution");

In [ ]:
sns.countplot(y="phys_health_consequence", data=data)

In [ ]:
data.phys_health_consequence.describe()

In [ ]:
sns.displot(x=data["phys_health_consequence"],hue="treatment",data=data,kde=False)
plt.title("Physical Health Conseuquence Distribution");

Ces deux variables représentent la présence ou non de conséquences si un employé venait à discuter de sa santé avec son manager. Mental_health_consequence correspond à la santé mentale et phys_health_consequence à la santé physique. On remarque qu'un "tabou" semble exister autour de la santé mentale au vu de la très forte proportion de Yes et Maybe. De plus, un manager plus "ouvert" sur les questions de santé mentale (réponse No) semblerait expliquer un moins grande prise de traitement. 

In [ ]:
sns.countplot(y="coworkers", data=data)

In [ ]:
data.coworkers.describe()

In [ ]:
sns.displot(x=data["coworkers"],hue="treatment",data=data,kde=False)
plt.title("Coworkers Distribution");

La variable représentant le fait de pouvoir discuter de santé mentale avec ses collègues semble avoir peu d'influence sur la prise de traitement, même si il y en a de légères. 

In [ ]:
sns.countplot(y="supervisor", data=data)

In [ ]:
data.supervisor.describe()

In [ ]:
sns.displot(x=data["supervisor"],hue="treatment",data=data,kde=False)
plt.title("Supervisor Distribution");

De même que pour les collègues, la variable représentant le fait de pouvoir discuter de santé mentale avec ses managers semble avoir peu d'influence sur la prise de traitement, même si il y en a de légères. 

In [ ]:
sns.countplot(y="mental_vs_physical", data=data)

In [ ]:
data.mental_vs_physical.describe()

In [ ]:
sns.displot(x=data["mental_vs_physical"],hue="treatment",data=data,kde=False)
plt.title("Mental VS Physical Distribution");

Cette variable répond à la question "pensez-vous que votre employeur prend autant au sérieux les problèmes de santé mentale que de santé physique ?". 

In [ ]:
sns.countplot(y="obs_consequence", data=data)

In [ ]:
data.obs_consequence.describe()

In [ ]:
sns.displot(x=data["obs_consequence"],hue="treatment",data=data,kde=False)
plt.title("Obs Consequence Distribution");

Cette variable permet d'observer les conséquences sur le plan professionel d'un collègue ayant parlé de santé mentale. Ceux pour lesquels une conséquence négative a eu lieu sont plus à même de prendre un traitement.

<h1> Classification Models </h1>

On veut déterminer si un particpant au sondage est traité ou non (suivi par un spécialiste ou non) selon ses réponses concernant son environnement de travail. Ainsi, nous cherchons à classer les participants en 2 catégories : ceux devant être traités et ceux n'en ayant pas besoin. C'est donc un problème de classification. 

In [ ]:
print(data.columns)

In [ ]:
#On encode afin de pouvoir appliquer les modèles 
from sklearn.preprocessing import LabelEncoder

lencode = LabelEncoder()
for columns in ['Gender', 'Country', 'state', 'self_employed', 'treatment', 'family_history', 'work_interfere', 'obs_consequence', 'no_employees', 'remote_work', 'tech_company', 'benefits', 'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave', 'mental_health_consequence', 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview', 'mental_vs_physical']:
    data[columns] = lencode.fit_transform(data[columns])

In [ ]:
data.describe()

In [ ]:
#on veut réduire la standard deviation de Age, Country, state
from sklearn.preprocessing import MaxAbsScaler, StandardScaler

data['Age'] = MaxAbsScaler().fit_transform(data[['Age']])
data['Country'] = StandardScaler().fit_transform(data[['Country']])
data['state'] = StandardScaler().fit_transform(data[['state']])

#on pourrait également diminuer celle de work_interfere et no_employees car > 1

In [ ]:
data.describe()

In [ ]:
Y=data['treatment']
X=data.drop(['treatment'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=0)

<h3> Logistic Regression </h3>

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_lr = LogisticRegression().fit(X_train, Y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay

In [ ]:
#pred = model_lr.predict(X_train)
Y_pred=model_lr.predict(X_test)

#print("Accuracy : ",accuracy_score(Y_train, pred))
print("Accuracy : ",accuracy_score(Y_test, Y_pred))
print("Precision Score : ", precision_score(Y_test, Y_pred))
print("Recall Score : ", recall_score(Y_test, Y_pred))
print("Confusion Matrix : \n", confusion_matrix(Y_test, Y_pred))

<h3> Random Forest </h3>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model_rf = RandomForestClassifier().fit(X_train, Y_train)

In [ ]:
Y_pred = model_rf.predict(X_test)

print("Accuracy : ",accuracy_score(Y_test, Y_pred))
print("Precision Score : ", precision_score(Y_test, Y_pred))
print("Recall Score : ", recall_score(Y_test, Y_pred))
print("Confusion Matrix : \n", confusion_matrix(Y_test, Y_pred))

<h3> Gaussian Naive Bayes Classifier </h3>

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
model_gnb = GaussianNB().fit(X_train, Y_train)

In [ ]:
Y_pred = model_gnb.predict(X_test)

print("Accuracy : ",accuracy_score(Y_test, Y_pred))
print("Precision Score : ", precision_score(Y_test, Y_pred))
print("Recall Score : ", recall_score(Y_test, Y_pred))
print("Confusion Matrix : \n", confusion_matrix(Y_test, Y_pred))

<h3> Nearest Neighbours </h3>

In [ ]:
from sklearn import neighbors

In [ ]:
model_knn = neighbors.KNeighborsClassifier().fit(X_train, Y_train)

In [ ]:
Y_pred = model_knn.predict(X_test)

print("Accuracy : ",accuracy_score(Y_test, Y_pred))
print("Precision Score : ", precision_score(Y_test, Y_pred))
print("Recall Score : ", recall_score(Y_test, Y_pred))
print("Confusion Matrix : \n", confusion_matrix(Y_test, Y_pred))

<h3> SVM Classifier </h3>

In [ ]:
from sklearn import svm

In [ ]:
model_svm = svm.SVC(kernel="linear").fit(X_train, Y_train)

In [ ]:
Y_pred = model_svm.predict(X_test)

print("Accuracy : ",accuracy_score(Y_test, Y_pred))
print("Precision Score : ", precision_score(Y_test, Y_pred))
print("Recall Score : ", recall_score(Y_test, Y_pred))
print("Confusion Matrix : \n", confusion_matrix(Y_test, Y_pred))

<h3> Gradient Boosting Classifier </h3>

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model_grad = GradientBoostingClassifier().fit(X_train, Y_train)

In [ ]:
Y_pred = model_grad.predict(X_test)

print("Accuracy : ",accuracy_score(Y_test, Y_pred))
print("Precision Score : ", precision_score(Y_test, Y_pred))
print("Recall Score : ", recall_score(Y_test, Y_pred))
print("Confusion Matrix : \n", confusion_matrix(Y_test, Y_pred))

Les deux modèles qui se démarquent sont Gradient Boosting et Random Forest (même Accuracy, Precision et Recall semblables). ce sont ces modèles qui ont la meilleure Accuracy. Il nous faut maintenant les départager.

On souhaite minimiser le nombre de "False Negative" dans la matrice de confusion, c'est à dire les personnes ayant réellement besoin d'un traitement, mais qui selon le modèle n'en ont pas besoin. De cette manière, nous pouvons maximiser le nombre de personnes qui ont besoin d'aide et qui sont traitées ou suivies. 

En suivant cette logique, le meilleur modèle à utiliser est donc Gradient Boosting.

<h1> Clustering </h1>

In [ ]:
#installation de KModes
#pip install KModes

In [ ]:
import numpy as np
from kmodes.kmodes import KModes

Nous retirons les ligne contenants des NaN, Ansi que certaines donnée perturbatrices : 'state' et 'self_employed' qui contient trop d'erreur

In [ ]:
data_droped = data.drop(['state', 'self_employed' ], axis=1)
data_cleaned = data_droped.dropna()
data_cleaned.columns

Nous avons choisi la méthode des K Modes pour l'apprentissage non supervisé, car nous n'utilisons que des variables qualitatives et cette méthode se prete bien à ce travail.

Nous avons choisi ici de construire 2 cluster, car on s'est interessés precédement à la prévisions de la variable treatment. Nous souhaitons voir si cette méthode de clustering nous permet d'identifier ces 2 groupes : les traités et les non traités.


In [ ]:
km = KModes(n_clusters=2, init='Huang', n_init=5, verbose=1)

clusters = km.fit_predict(data_cleaned)

Selon les executions, les resultats peuvent differer, mais le modele me renvoie quasiment toujours un centroid où treatment = yes et un centroid où treatment = no

In [ ]:
print(km.cluster_centroids_)

In [ ]:
plt.figure(figsize=(10,6))
plt.title("treatment Distribution");
sns.scatterplot(data_cleaned.loc[:,"treatment"], c = km.labels_, )

On parvient bien à identifier 2 clusters, l'un avec les personnes traités, l'autres avec les personnes non traités

In [ ]:
sns.displot(x=data_cleaned["family_history"],hue=km.labels_,data=data_cleaned,kde=False)
plt.title("Family History Distribution");

In [ ]:
sns.displot(x=data_cleaned["work_interfere"],hue=km.labels_,data=data_cleaned,kde=False)
plt.title("work_interfere");

In [ ]:
sns.displot(x=data_cleaned["no_employees"],hue=km.labels_,data=data_cleaned,kde=False)
plt.title("no_employees");

In [ ]:
sns.displot(x=data_cleaned["care_options"],hue=km.labels_,data=data_cleaned,kde=False)
plt.title("care_options");

On peut observer des comportement similaire a la comparaison des variables selon treatment


Après avoir tenté d'utiliser la méthode avec plus de cluster, n'arrivant pas à identifier des comportement clairs et interprétables, nous avons décidé d'en rester là avec 2 clusters.